# 医疗文档图像文字识别演示

> 🏥 使用PaddleOCR从医疗文档PNG图像中提取文本并保存为CSV

**版本**: v1.2.0 (实现本地测试运行环境) | **更新时间**: 2025-08-17

## 🎯 功能特性
- 📄 支持医疗文档图像文字识别
- 🤖 使用PaddleOCR高精度识别引擎
- 📊 自动生成结构化CSV报告
- 🖼️ 支持多种图像格式输入
- 💡 简单易用的交互界面
- 🏠 完整本地开发环境支持

## 🚀 使用说明

### Colab环境
1. 运行环境检查和依赖安装
2. 上传医疗文档图像
3. 执行OCR文字识别
4. 下载CSV结果文件

### 本地环境
```bash
# 一键启动 (推荐)
./start_local.sh

# 手动启动
source venv/bin/activate && jupyter notebook
```

---
*使用 Claude Code 开发，支持 Google Colab 和本地运行 🚀*

In [ ]:
# ================================
# 环境检查和基础设置
# ================================

import warnings
warnings.filterwarnings("ignore")

def check_environment():
    """检查运行环境并显示系统信息"""
    print("🔍 检查运行环境...")
    
    # 检查是否在Colab环境
    try:
        import google.colab  # noqa: F401
        print("✅ 运行在Google Colab")
        in_colab = True
    except ImportError:
        print("ℹ️ 运行在本地环境")
        in_colab = False
    
    # 检查GPU
    try:
        import torch
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"✅ 计算设备: {device}")
        if device == 'cuda':
            print(f"✅ GPU型号: {torch.cuda.get_device_name(0)}")
            print(f"✅ GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    except ImportError:
        print("ℹ️ PyTorch未安装，使用CPU模式")
    
    return in_colab

# 运行环境检查
in_colab = check_environment()

In [ ]:
# ================================
# 安装必要的依赖包
# ================================

def install_dependencies():
    """安装项目所需的依赖包"""
    print("📦 安装医疗OCR项目依赖...")
    
    import subprocess
    import sys
    
    # 核心依赖包列表
    packages = [
        'paddlepaddle',
        'paddleocr',
        'pandas',
        'pillow',
        'opencv-python',
        'tqdm',
        'gradio'
    ]
    
    for package in packages:
        try:
            if package == 'opencv-python':
                import cv2  # noqa: F401
                print(f"✅ {package} 已安装")
            elif package == 'pillow':
                from PIL import Image  # noqa: F401
                print(f"✅ {package} 已安装")
            else:
                __import__(package.replace('-', '_'))
                print(f"✅ {package} 已安装")
        except ImportError:
            print(f"📥 安装 {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    
    print("✅ 所有依赖安装完成!")

# 安装依赖
install_dependencies()

In [ ]:
# ================================
# 导入必要的库
# ================================

try:
    import pandas as pd  # type: ignore
    from PIL import Image
    from tqdm import tqdm
    from paddleocr import PaddleOCR  # type: ignore
    import gradio as gr  # type: ignore
    print("📚 所有库导入成功!")
except ImportError as e:
    print(f"❌ 库导入失败: {e}")
    print("💡 请先运行依赖安装单元格")

In [ ]:
# ================================
# 医疗OCR核心功能类
# ================================

class MedicalOCRProcessor:
    def __init__(self):
        """初始化医疗OCR处理器"""
        print("🏥 初始化医疗OCR处理器...")
        
        # 检查GPU可用性
        try:
            import torch
            use_gpu = torch.cuda.is_available()
            gpu_info = f"GPU可用: {use_gpu}"
            if use_gpu:
                gpu_info += f" (设备: {torch.cuda.get_device_name(0)})"
            print(f"⚡ {gpu_info}")
        except ImportError:
            use_gpu = False
            print("ℹ️ PyTorch未安装，使用CPU模式")
        
        # 初始化PaddleOCR，支持中英文
        # 新版PaddleOCR会自动检测并使用GPU（如果可用）
        self.ocr = PaddleOCR(use_angle_cls=True, lang='ch')
        
        print("✅ OCR引擎初始化完成")
    
    def extract_text_from_image(self, image_path):
        """从图像中提取文字"""
        try:
            # 使用PaddleOCR识别文字（移除cls参数以兼容新版API）
            result = self.ocr.ocr(image_path)
            
            # 提取文字内容
            extracted_texts = []
            
            # 确保result不为空且包含有效数据
            if result and len(result) > 0 and result[0] is not None:
                for line in result[0]:
                    # 确保line有足够的元素且结构正确
                    if (line and len(line) >= 2 and 
                        line[1] and len(line[1]) >= 2 and 
                        isinstance(line[1][0], str)):
                        
                        text = line[1][0]
                        confidence = line[1][1]
                        
                        # 只添加非空文字
                        if text and text.strip():
                            extracted_texts.append({
                                'text': text.strip(),
                                'confidence': confidence
                            })
            
            if not extracted_texts:
                print("⚠️ 未检测到任何文字内容")
            else:
                print(f"✅ 成功识别 {len(extracted_texts)} 行文字")
            
            return extracted_texts
        
        except Exception as e:
            print(f"❌ 图像处理失败: {str(e)}")
            import traceback
            print(f"详细错误信息: {traceback.format_exc()}")
            return []
    
    def process_single_image(self, image_path):
        """处理单个图像文件"""
        import os
        print(f"📄 处理图像: {os.path.basename(image_path)}")
        
        # 提取文字
        extracted_texts = self.extract_text_from_image(image_path)
        
        # 整理结果
        results = []
        for i, item in enumerate(extracted_texts):
            results.append({
                'file_name': os.path.basename(image_path),
                'line_number': i + 1,
                'extracted_text': item['text'],
                'confidence': round(item['confidence'], 4)
            })
        
        return results
    
    def process_multiple_images(self, image_paths):
        """批量处理多个图像文件"""
        all_results = []
        
        print(f"📊 开始批量处理 {len(image_paths)} 个图像文件...")
        
        for image_path in tqdm(image_paths, desc="处理进度"):
            results = self.process_single_image(image_path)
            all_results.extend(results)
        
        return all_results
    
    def save_results_to_csv(self, results, output_path):
        """保存结果到CSV文件"""
        if not results:
            # 如果没有结果，创建空的DataFrame
            df = pd.DataFrame(columns=['file_name', 'line_number', 'extracted_text', 'confidence'])
        else:
            df = pd.DataFrame(results)
        
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"💾 结果已保存到: {output_path}")
        return df

# 初始化OCR处理器
ocr_processor = MedicalOCRProcessor()

In [ ]:
# ================================
# 创建示例医疗文档（用于演示）- 修复字体和绘制问题
# ================================

def create_sample_medical_document():
    """创建示例医疗文档图像用于演示"""
    from PIL import Image, ImageDraw, ImageFont
    import os
    
    # 创建示例图像 - 使用更大尺寸和更好对比度
    img = Image.new('RGB', (1000, 800), color='white')
    draw = ImageDraw.Draw(img)
    
    # 添加边框增加文档感
    draw.rectangle([(20, 20), (980, 780)], outline='black', width=2)
    
    # 使用英文内容确保字体兼容性
    sample_text = [
        "MEDICAL REPORT",
        "Hospital: XX City People's Hospital",
        "Patient: Zhang San",
        "Gender: Male    Age: 45",
        "Department: Cardiology", 
        "Doctor: Dr. Li",
        "Diagnosis: Hypertension, Diabetes",
        "Prescription:",
        "1. Antihypertensive 10mg daily",
        "2. Antidiabetic 5mg twice daily",
        "Doctor Signature: Dr. Li",
        "Date: 2025-08-17"
    ]
    
    # 获取更大更清晰的字体
    font = None
    font_size = 32  # 增大字体
    
    try:
        # 尝试使用系统字体
        font_paths = [
            '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
            '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf'
        ]
        
        for font_path in font_paths:
            if os.path.exists(font_path):
                try:
                    font = ImageFont.truetype(font_path, font_size)
                    print(f"✅ 使用字体: {font_path}")
                    break
                except Exception as e:
                    print(f"⚠️ 字体加载失败: {font_path} - {e}")
                    continue
    except Exception as e:
        print(f"⚠️ 字体加载过程异常: {e}")
    
    # 使用默认字体作为备用
    if font is None:
        try:
            font = ImageFont.load_default()
            print("✅ 使用PIL默认字体")
        except:
            print("⚠️ 使用系统默认字体")
            font = None
    
    # 绘制标题
    title_font_size = 40
    try:
        if font:
            title_font = ImageFont.truetype(font.path, title_font_size) if hasattr(font, 'path') else font
        else:
            title_font = None
    except:
        title_font = font
    
    # 绘制文本内容
    y_position = 60
    line_height = 50
    
    for i, text in enumerate(sample_text):
        try:
            # 第一行标题居中加粗
            if i == 0:
                # 计算居中位置
                if title_font:
                    bbox = draw.textbbox((0, 0), text, font=title_font)
                    text_width = bbox[2] - bbox[0]
                else:
                    text_width = len(text) * 12  # 估算宽度
                x_position = (1000 - text_width) // 2
                
                # 绘制标题
                if title_font:
                    draw.text((x_position, y_position), text, fill='black', font=title_font)
                else:
                    draw.text((x_position, y_position), text, fill='black')
                print(f"✅ 绘制标题: {text}")
                
                # 添加下划线
                draw.line([(x_position, y_position + 45), (x_position + text_width, y_position + 45)], 
                         fill='black', width=2)
                y_position += 20  # 标题后额外间距
            else:
                # 普通文本左对齐
                x_position = 60
                if font:
                    draw.text((x_position, y_position), text, fill='black', font=font)
                else:
                    draw.text((x_position, y_position), text, fill='black')
                print(f"✅ 绘制文本: {text}")
            
        except Exception as e:
            print(f"⚠️ 文字绘制失败: {e} - 文本: {text}")
            # 简单备用绘制
            try:
                x_pos = 60 if i > 0 else 200
                draw.text((x_pos, y_position), text, fill='black')
                print(f"✅ 备用方式绘制: {text}")
            except Exception as e2:
                print(f"❌ 备用绘制也失败: {e2}")
        
        y_position += line_height
    
    # 添加一些装饰元素增加真实感
    try:
        # 添加医院LOGO占位符
        draw.rectangle([(60, 100), (160, 160)], outline='gray', width=1)
        draw.text((80, 125), "LOGO", fill='gray')
        
        # 添加签名线
        draw.line([(700, 650), (950, 650)], fill='black', width=1)
        draw.text((700, 660), "Signature", fill='gray')
        
        print("✅ 装饰元素添加完成")
    except Exception as e:
        print(f"⚠️ 装饰元素添加失败: {e}")
    
    # 保存示例图像
    sample_path = 'sample_medical_document.png'
    img.save(sample_path, quality=95, optimize=True)
    print(f"📄 创建示例医疗文档: {sample_path}")
    
    # 验证文件创建
    if os.path.exists(sample_path):
        file_size = os.path.getsize(sample_path)
        print(f"✅ 文件创建成功，大小: {file_size} 字节")
    else:
        print("❌ 文件创建失败")
    
    return sample_path

def translate_to_english(chinese_text):
    """将中文文本转换为英文（备用方案）"""
    translations = {
        "医院名称：XX市人民医院": "Hospital: XX City People's Hospital",
        "患者姓名：张三": "Patient Name: Zhang San",
        "性别：男    年龄：45岁": "Gender: Male    Age: 45",
        "科室：心血管内科": "Department: Cardiology",
        "主治医师：李医生": "Doctor: Dr. Li",
        "诊断：高血压、糖尿病": "Diagnosis: Hypertension, Diabetes",
        "处方：": "Prescription:",
        "1. 降压药 10mg 每日一次": "1. Antihypertensive 10mg once daily",
        "2. 降糖药 5mg 每日两次": "2. Antidiabetic 5mg twice daily",
        "医生签名：李医生": "Doctor Signature: Dr. Li",
        "日期：2024-08-17": "Date: 2024-08-17"
    }
    return translations.get(chinese_text, chinese_text)

# 创建示例文档
print("🎨 创建示例医疗文档...")
sample_doc = create_sample_medical_document()

# 显示示例图像
try:
    from IPython.display import Image as IPImage, display  # type: ignore # noqa: F401
    display(IPImage(sample_doc))
except ImportError:
    # 在非Jupyter环境中的备用方案
    print(f"✅ 示例文档已创建: {sample_doc}")
    print("💡 在Jupyter/Colab环境中会自动显示图像")

In [ ]:
# ================================
# 演示OCR功能
# ================================

def demo_ocr_functionality():
    """演示OCR功能"""
    print("🚀 开始演示医疗OCR功能...")
    
    # 处理示例文档
    results = ocr_processor.process_single_image(sample_doc)
    
    # 显示识别结果
    print("\n📊 文字识别结果:")
    print("-" * 60)
    
    for result in results:
        print(f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})")
    
    # 保存结果到CSV
    csv_path = 'ocr_results_demo.csv'
    df = ocr_processor.save_results_to_csv(results, csv_path)
    
    print(f"\n📈 共识别出 {len(results)} 行文字")
    print(f"📄 结果已保存到 CSV 文件: {csv_path}")
    
    # 显示CSV内容预览
    print("\n📋 CSV文件预览:")
    print(df.to_string(index=False))
    
    return df, csv_path

# 运行演示
demo_df, demo_csv = demo_ocr_functionality()

In [ ]:
# ================================
# 创建交互式界面
# ================================

def create_gradio_interface():
    """创建Gradio交互界面"""
    
    def process_uploaded_image(image):
        """处理上传的图像"""
        if image is None:
            return "请上传图像文件", None
        
        try:
            # 保存临时文件
            temp_path = "temp_uploaded_image.png"
            Image.fromarray(image).save(temp_path)
            
            # 处理图像
            results = ocr_processor.process_single_image(temp_path)
            
            if not results:
                return "未识别到文字内容", None
            
            # 生成结果文本
            result_text = "📊 识别结果:\n\n"
            for i, result in enumerate(results, 1):
                result_text += f"{i:2d}. {result['extracted_text']} (置信度: {result['confidence']:.3f})\n"
            
            # 保存CSV文件
            csv_path = "ocr_results.csv"
            ocr_processor.save_results_to_csv(results, csv_path)
            
            result_text += f"\n✅ 共识别 {len(results)} 行文字"
            result_text += f"\n💾 结果已保存到: {csv_path}"
            
            return result_text, csv_path
            
        except Exception as e:
            return f"❌ 处理失败: {str(e)}", None
    
    # 创建界面
    interface = gr.Interface(
        fn=process_uploaded_image,
        inputs=[
            gr.Image(label="上传医疗文档图像", type="numpy")
        ],
        outputs=[
            gr.Textbox(label="识别结果", lines=15),
            gr.File(label="下载CSV结果文件")
        ],
        title="🏥 医疗文档OCR识别系统",
        description="上传医疗文档图像，自动识别其中的文字内容并生成CSV报告。支持中英文混合识别。",
        examples=[
            [sample_doc]
        ],
        theme=gr.themes.Soft()
    )
    
    return interface

# 创建并启动界面
print("🌐 创建交互界面...")
interface = create_gradio_interface()

if interface:
    print("✅ 界面创建成功!")
    print("🚀 启动界面服务...")
    interface.launch(
        share=True,
        debug=True,
        show_error=True
    )
else:
    print("❌ 界面创建失败")

In [ ]:
# ================================
# 项目总结和使用指南
# ================================

def show_project_summary():
    """显示项目总结和使用指南"""
    
    summary = """
🎉 医疗文档OCR识别演示项目运行完成!

📋 项目功能总结:
✅ 医疗文档图像文字识别
✅ 中英文混合文字识别
✅ 批量图像处理支持
✅ CSV格式结果导出
✅ 置信度评估
✅ 用户友好的交互界面

🛠️ 技术特性:
• 基于PaddleOCR高精度识别引擎
• 支持GPU加速处理
• 自动角度检测和矫正
• 结构化数据输出

📖 使用场景:
• 医疗处方识别
• 病历文档数字化
• 检查报告提取
• 医疗档案管理

🔗 项目地址:
GitHub: https://github.com/zhurong2020/mcr
Colab: 当前notebook链接

👨‍⚕️ 适用用户:
• 医疗工作者
• 数据分析师
• 研究人员
• 开发者

⚠️ 使用说明:
1. 确保图像清晰可读
2. 支持PNG、JPG等常见格式
3. 批量处理时注意文件大小
4. 结果仅供参考，重要信息请人工核验

💡 技术支持:
如有问题或建议，欢迎反馈交流！
    """
    
    print(summary)

# 显示项目总结
show_project_summary()